# HW 2 Excel with Python Part 2

Wrangling .csv files using Python

## Preliminaries

In [1]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [2]:
import pandas as pd
from glob import glob
import glob
from openpyxl import Workbook
import openpyxl 
from openpyxl import load_workbook
import shutil
from pathlib import Path
from openpyxl.utils.dataframe import dataframe_to_rows

In [3]:
%matplotlib inline

### Create blank Excel file

Create blank Excel file named BCM.xlsx using the openpyxl library and Python

In [15]:
# Create excel file using openpyxl Workbook library
wb = Workbook()

# Save workbook with BCM filename
wb.save("output/BCM.xlsx")

### Insert csv File Contents into new sheet in BCM file

For this I used pandas and pathlib. I also used file globbing and a loop and kept a list of the csv files to use in the next formula block.

In [16]:
# Set the input csv folder and Excel output file paths
csvs_path = Path.cwd() / 'data' 
excel_path = Path.cwd() / 'output' / 'BCM.xlsx'

# create some variables of files and other parameters
filename = 'output/BCM.xlsx' 
separator =','
# create list of csv filenames= sheets for formula block below
sheet_list = []

# Use ExcelWriter to add all CSVs as separate sheets with sheet name = csv filename  
# Loop over all the csv files to add each csv file data to each Excel sheet  
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
    for csv_file in csvs_path.glob('*.csv'):
        sheet_name = csv_file.stem[:]  # current excel sheet name 
        sheet_list.append(sheet_name)  # create list of csv files for later

        # Create dataframe of data in current csv - add headers with 'names', capture first 3 cols
        # parsing the first column with pandas date_format but doesn't end up keeping time
        df = pd.read_csv(
            csv_file,
            sep=',',
            parse_dates = [0], 
            date_format = '%Y-%m-%d-%H-%M',
            encoding='utf-8',
            index_col=0,
            usecols=[0, 1, 2],
            header=None,
            names=['datetime', 'scale', 'temperature']
        )
        # print(sheet_name) # used for testing
        # print(df) # used to check contents of dataframes in testing
        
        # add dataframe to current excel sheet
        df.to_excel(writer, sheet_name=sheet_name, index=True)

        # save excel output file before next loop
        wb.save(excel_path)
        
# Only remove default sheet if there are other sheets
if 'Sheet' in wb.sheetnames and len(wb.sheetnames) > 1:
    del wb['Sheet']
        
print(sheet_list)

['BCM-N-tCenter-Medium', 'BCM-E-tLeft-Deep', 'BCM-E-tRight-Medium', 'BCM-E-tRight-Deep', 'BCM-N-tLeft-Deep', 'BCM-E-tCenter-Medium', 'BCM-E-tCenter-Shallow', 'BCM-N-tRight-Shallow', 'BCM-N-tRight-Medium', 'BCM-E-tCenter-Deep', 'BCM-N-tRight-Deep', 'BCM-N-tLeft-Medium', 'BCM-N-tCenter-Deep', 'BCM-E-tLeft-Medium']


### Add Formulas to Each Sheet

a.	Reopened the modified Excel file with openpyxl.
b.	Loop over the appropriate sheets and create the formulas and adjacent labels.
c.	The openpyxl Tutorial at https://openpyxl.readthedocs.io/en/stable/tutorial.html has some useful information.
d.	Save the Excel file.

add some simple formulas to each sheet showing the minimum, maximum, and average of the temperature values. The labels should be in G2:G4 and the formulas in H2:H4. Notice, they want actual Excel formulas in H2:H4, not just computed values. Here’s a screenshot to make this clearer. Notice the nice cell formatting in column H. In addition, compute the minimum and maximum of the datetime field in rows 6 and 7.

In [18]:
# Add labels and formulas to each sheet

# Load the workbook and delete first empty sheet
wb = load_workbook(filename)

# Only remove default sheet if there are other sheets
if 'Sheet' in wb.sheetnames and len(wb.sheetnames) > 1:
    del wb['Sheet']
        
# Loop through each worksheet and add labels and formulas for each sheet
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]

    # Add labels to G2: G7 for min, mean, and max temps, and min and max dates
    ws['G2'] = 'min_temp'
    ws['G3'] = 'max_temp'
    ws['G4'] = 'mean_temp'
    ws['G6'] = 'min_date'
    ws['G7'] = 'max_date'

    # Find max row to know how many rows to sum in next formulas
    last_row = ws.max_row
 
    # Add formulas to H2:H4 using Excel formulas
    ws['H2'] = f"=min(C2:C{last_row})"
    ws['H3'] = f"=max(C2:C{last_row})"
    ws['H4'] = f"=average(C2:C{last_row})"

    # Convert 'date/time' in string to date / time value for computation, then format as date/time 
    # unfortunately time element isn't working as desired even though read in and parsed as such
    # in block above. I think it's because it replaced the index column.
    ws['H6'] = f"=min(datevalue(A2:A{last_row}))"
    ws['H6'].number_format = 'yyyy-mm-dd hh:mm'
    ws['H7'] = f"=max(datevalue(A2:A{last_row}))"
    ws['H7'].number_format = 'yyyy-mm-dd hh:mm'
 
wb.save(filename)
 

### Hints

Used these above in coding for capturing file path, creating excel file, reading csv, using workbook and sheets. 
a.	The pathlib library is very useful for working with paths and filenames.
b.	Remember, cell addresses and even cell formulas are just strings. Python f-strings are quite handy for building strings based on variable values.
c.	There’s a very simple way to determine the row number of the last row of data in a file using openpyxl.
d.	In openpyxl, many collections (such as worksheets) are just Python lists.
e.	You must heavily comment your code to explain what you are doing.